In [1]:
!git clone https://github.com/benihime91/leaf-disease-classification-kaggle.git
!wandb login a74f67fd5fae293e301ea8b6710ee0241f595a63

Cloning into 'leaf-disease-classification-kaggle'...
remote: Enumerating objects: 385, done.
remote: Counting objects: 100% (385/385), done.
remote: Compressing objects: 100% (291/291), done.
remote: Total 1161 (delta 176), reused 286 (delta 86), pack-reused 776
Receiving objects: 100% (1161/1161), 39.78 MiB | 30.01 MiB/s, done.
Resolving deltas: 100% (644/644), done.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [2]:
import sys
sys.path.append('../input/timmmodels/pytorch-image-models/')
sys.path.append('leaf-disease-classification-kaggle/')

In [3]:
import pytorch_lightning as pl
import torch
from torch import nn, optim
import timm
import albumentations as A
from albumentations.pytorch import ToTensorV2

from fastai.torch_core import apply_init
from functools import partial
import wandb

from src.core import *
from src.lightning.core import *
from src.layers import *
from src.mixmethods import *

In [4]:
seed = seed_everything(42)
idx  = generate_random_id()

In [5]:
# configure the training paramters/job

config = {
    "random_seed": seed,
    "unique_idx" : idx,
    
    # name of the Wandb project name
    "project_name" : "kaggle-leaf-disease-v2",
    
    # current fold of data to take as validation, other folds are used as
    # train data
    "curr_fold" : 0 ,
    # path to where the images are present
    "image_dir" : "../input/cassava-leaf-disease-classification/train_images/",
    # path to the stratified 5 folds csv data for cassava
    "csv_path"  : "leaf-disease-classification-kaggle/data/stratified-data-5folds.csv",
    
    # settings for the model architecture
    "encoder"   : "resnext50_32x4d",
    "activation": nn.ReLU(inplace=True),
    "image_dims": 512,
    
    # settings for the training job
    "num_epochs": 30,
    "opt_func"  : partial(optim.Adam, weight_decay=0, eps=1e-08, betas=(0.9, 0.999)),
    "scheduler" : partial(optim.lr_scheduler.MultiStepLR, milestones=[16, 26], gamma=0.1),
    "mixmethod" : partial(SnapMix, alpha=0.5, conf_prob=1.0, mid_level=True),
    "loss_func" : LabelSmoothingCrossEntropy(),
    # when to call scheduler.step() -- "step" or "epoch"
    "scheduler_step" : "epoch",
    "batch_size": 32,
    # initial learning rate for the optimizers
    "init_lr": 2e-03,
    "lr_mult": 100, 
}

# Albumentations augmentations for train/ valid data
TRAIN_AUGS = A.Compose([
    A.OneOf([
        A.RandomResizedCrop(config["image_dims"], config["image_dims"]), 
        A.CenterCrop(config["image_dims"], config["image_dims"])], 
    p=0.7),
    A.Resize(config["image_dims"], config["image_dims"], p=1.0),
    A.HorizontalFlip(),
    A.OneOf([A.ShiftScaleRotate(), A.Flip(), A.Transpose()], p=0.8),
    A.RandomBrightnessContrast(p=0.5),
    A.Normalize(p=1.0),
    ToTensorV2(p=1.0)
])
    
VALID_AUGS = A.Compose([
    A.Resize(config["image_dims"], config["image_dims"], p=1.0), 
    A.Normalize(p=1.0),
    ToTensorV2(p=1.0),
])

MODEL_SAVE_PATH = f"{config['encoder']}-fold={config['curr_fold']}-{idx}.pt"

In [6]:
# initate the model architecture
# for snapmix we will call BasicTransferLearningModel class to init a model
# suitable for snapmix, we can also use TransferLearningModel class to init
# a model similar to the model created by the fast.ai cnn_learner func

encod = timm.create_model(config["encoder"], pretrained=True)
model = SnapMixTransferLearningModel(encod, len(idx2lbl), -2, act=config["activation"])

# init the weights of the final untrained layer
apply_init(model.fc, torch.nn.init.kaiming_normal_)
apply_init(model.mcls, torch.nn.init.kaiming_normal_)

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/resnext50_32x4d_ra-d733960d.pth" to /root/.cache/torch/hub/checkpoints/resnext50_32x4d_ra-d733960d.pth


In [7]:
print(model)

SnapMixTransferLearningModel(
  (encoder): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act1): ReLU(inplace=True)
        (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act2): ReLU(inplace=True)
        (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running

In [8]:
# init the LightingDataModule + LightningModule
dm = CassavaLightningDataModule(config["csv_path"], config["image_dir"], curr_fold=config["curr_fold"], 
                                train_augs=TRAIN_AUGS, valid_augs=VALID_AUGS, bs=config["batch_size"], num_workers=3)

litModel = LightningCassava(model, opt_func=config["opt_func"], lr=config["init_lr"], lr_mult=config["lr_mult"],
                            step_after=config["scheduler_step"], scheduler=config["scheduler"],
                            loss_func=config["loss_func"], mixmethod=config["mixmethod"])

Using functools.partial(<class 'src.mixmethods.SnapMix'>, alpha=0.5, conf_prob=1.0, mid_level=True)
Uses LabelSmoothingCrossEntropy()


In [ ]:
# initialize pytorch_lightning Trainer + Callbacks
callbacks = [pl.callbacks.LearningRateMonitor("step"), WandbImageClassificationCallback(dm)]
chkpt_callback = pl.callbacks.ModelCheckpoint(monitor="valid/acc", save_top_k=1, mode='max')
wb_logger  = pl.loggers.WandbLogger(project=config["project_name"])

trainer = pl.Trainer(gpus=1, gradient_clip_val=0.1, precision=16, max_epochs=config["num_epochs"],
                     callbacks=callbacks, checkpoint_callback=chkpt_callback, logger=wb_logger, deterministic=True,
                     default_root_dir="/kaggle/working/", log_every_n_steps=20, terminate_on_nan=True)

In [ ]:
# start learning_rate finder to find optimum starting Lr
lr_finder = trainer.tuner.lr_find(litModel, datamodule=dm)

fig = lr_finder.plot(suggest=True)
fig.show()

In [ ]:
# modify the initial learning rate 
litModel.hparams['lr'] = 1e-03

# start the training job
trainer.fit(litModel, datamodule=dm)

In [ ]:
results = trainer.test(litModel, datamodule=dm)